In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from scipy import sparse
from implicit.als import AlternatingLeastSquares

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from dotenv import load_dotenv
load_dotenv()
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px 

from models.track import Track
from models.user import User
from models.elasticLink import ElasticLink

c:\Users\sebas\Proyectos\MusicalMatch\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\sebas\AppData\Local\Temp\ipykernel_29748\1050907235.py:18: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# load env variables
load_dotenv()
elastic_endpoint = os.getenv("ELASTIC_HOST")
elastic_api_key = os.getenv("API_KEY")
spotify_client_id = os.getenv("SPOTIFY_CLIENT_ID")
spotify_client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

# elasticsearch link 
es = ElasticLink(elastic_endpoint, elastic_api_key)

# spotify
os.environ["SPOTIPY_CLIENT_ID"] = spotify_client_id
os.environ["SPOTIPY_CLIENT_SECRET"] = spotify_client_secret
os.environ['SPOTIPY_REDIRECT_URI']='http://localhost:8888/callback'
client_credentials_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# needed premisions for each user
scope = 'user-top-read user-read-recently-played playlist-read-private'

In [3]:
u = User('test')
top = u.getTopTracks(es, scope)

In [4]:
u.userMetaVector

array([ 0.68646667,  0.73523333,  4.4       , -6.0357    ,  0.5       ,
        0.09750333,  0.13819057,  0.05981459,  0.17771   ,  0.46110667,
        1.27169133,  2.11128433,  3.96666667])

Simulo distintos usuarios

In [40]:
tsne_pipeline = Pipeline([('scaler', StandardScaler(with_mean=False)),
                          ('tsne', TSNE(n_components=2, random_state=0, perplexity=10, init='random'))])
track_embedding = tsne_pipeline.fit_transform(X)

projection = pd.DataFrame(columns=['x', 'y'], data=track_embedding)
projection['name'] = [track.name for track in total_tracks]
projection['type'] = ['songs' for t in range(len(total_tracks)-len(test_users))] + ['test'] * len(test_users)

fig = px.scatter(projection, x='x', y='y', color='type', hover_data=['x', 'y', 'name', 'type'])
fig.update_layout(title='T-SNE projection of the Top Tracks')
fig.show()